In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [2]:
X = np.arange(8).reshape(4, 2)
from sklearn.preprocessing import PolynomialFeatures
poly= PolynomialFeatures(2, include_bias = False)
poly.fit_transform(X)

array([[ 0.,  1.,  0.,  0.,  1.],
       [ 2.,  3.,  4.,  6.,  9.],
       [ 4.,  5., 16., 20., 25.],
       [ 6.,  7., 36., 42., 49.]])

### Importing Required Libraries

In [3]:
!pip install  pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=a1ddfb01301506b56c17704b82a9f909ecfdc0ca5e8b05d6310755455ad11b21
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=be41ae13b0db7a128da02fac4513fa5ed1d343c9def181153e8ec2183085526e
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyLDAvis sklearn


In [4]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from nltk.corpus import stopwords 
import re 
import gensim
from gensim.utils import simple_preprocess
from gensim.models import LdaMulticore
import gensim.corpora as corpora
from pprint import pprint
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [12]:
data = pd.read_csv("content/papers.csv")

FileNotFoundError: ignored

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [ ]:
data = data.drop(columns=['id',"event_type","pdf_name"],axis=1).sample(100)
data.head()

,year,title,abstract,paper_text
4742,2014,A Safe Screening Rule for Sparse Logistic Regr...,The l1-regularized logistic regression (or spa...,A Safe Screening Rule for Sparse Logistic Regr...
1151,1989,The CHIR Algorithm for Feed Forward Networks w...,Abstract Missing,516\n\nGrossman\n\nThe CHIR Algorithm for Feed...
1565,2003,Impact of an Energy Normalization Transform on...,Abstract Missing,Impact of an Energy Normalization\nTransform o...
5045,2014,Self-Paced Learning with Diversity,Self-paced learning (SPL) is a recently propos...,Self-Paced Learning with Diversity\n\nLu Jiang...
2774,2008,Transfer Learning by Distribution Matching for...,We address the problem of learning classifiers...,Transfer Learning by Distribution Matching\nfo...


### Data Cleaning

In [ ]:
def clean(text):
    text = text.lower()
    text = re.sub("[^\w\d]"," ",text)
    return text

<>:3: DeprecationWarning: invalid escape sequence \w
<>:3: DeprecationWarning: invalid escape sequence \w
<>:3: DeprecationWarning: invalid escape sequence \w
/tmp/ipykernel_19/1318172863.py:3: DeprecationWarning: invalid escape sequence \w
  text = re.sub("[^\w\d]"," ",text)


In [ ]:
data['cleaned'] = data['paper_text'].apply(clean)

In [ ]:
data[["paper_text","cleaned"]].head()

,paper_text,cleaned
4742,A Safe Screening Rule for Sparse Logistic Regr...,a safe screening rule for sparse logistic regr...
1151,516\n\nGrossman\n\nThe CHIR Algorithm for Feed...,516 grossman the chir algorithm for feed for...
1565,Impact of an Energy Normalization\nTransform o...,impact of an energy normalization transform on...
5045,Self-Paced Learning with Diversity\n\nLu Jiang...,self paced learning with diversity lu jiang1 ...
2774,Transfer Learning by Distribution Matching\nfo...,transfer learning by distribution matching for...


### After cleaning has done , Topic modeling requires preparing data according to algorithm like in deep learning we process data according to keras or torch

In [ ]:
STOPWORDS = stopwords.words('english')
STOPWORDS.extend(['from','subject','re','edu','use'])

In [ ]:

def sent_to_words(text):
    for sent in text:
        yield(gensim.utils.simple_preprocess(str(sent),deacc=True)) # Deacc remove punctuations !

#### what does simple_preprocess do ?
#### => it converts text to token that cannot be process further 
#### Link to gensim Simple Preprocess : https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html

In [ ]:
new_text  = sent_to_words(["when nothing is going right go left",
                           "My Stomach is FLAT. The L is just Silent",
                           "I am not Lazy , I am just on Energy Saving mode"])

In [ ]:
list(new_text)

[['when', 'nothing', 'is', 'going', 'right', 'go', 'left'],
 ['my', 'stomach', 'is', 'flat', 'the', 'is', 'just', 'silent'],
 ['am', 'not', 'lazy', 'am', 'just', 'on', 'energy', 'saving', 'mode']]

In [ ]:


def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in STOPWORDS] for doc in texts]

In [ ]:
text = data['cleaned'].values.tolist()
data_words = sent_to_words(list(text))
data_words = remove_stopwords(data_words)

In [ ]:
data_words[:1][0][:5]

['safe', 'screening', 'rule', 'sparse', 'logistic']

In [ ]:
len(data_words)

100

In [ ]:
len(data_words[0])

2382

### Now after setting it according to Algorithm , we will create a corpus to be fit on algorithm

In [ ]:
# Create Dictionary 
id2word = corpora.Dictionary(data_words)

# Create Corpus , a copy or you can say 
texts = data_words 

# TERM Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
# Now see The Difference

print("Normal Data Words: ",data_words[:1][0][:5])
print("Created Corpus : ",corpus[:1][0][:5])

Normal Data Words:  ['safe', 'screening', 'rule', 'sparse', 'logistic']
Created Corpus :  [(0, 1), (1, 1), (2, 1), (3, 2), (4, 1)]


### Time to Train Models

In [ ]:
num_topics = 10 #Just Assuming


lda_model  = LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

In [ ]:
pprint(lda_model.print_topics())

[(0,
  '0.006*"learning" + 0.005*"model" + 0.005*"data" + 0.005*"algorithm" + '
  '0.004*"time" + 0.004*"neural" + 0.004*"function" + 0.004*"using" + '
  '0.003*"training" + 0.003*"set"'),
 (1,
  '0.006*"algorithm" + 0.005*"log" + 0.005*"learning" + 0.004*"neural" + '
  '0.004*"using" + 0.004*"model" + 0.004*"data" + 0.003*"one" + 0.003*"set" + '
  '0.003*"function"'),
 (2,
  '0.005*"learning" + 0.004*"model" + 0.004*"data" + 0.004*"one" + '
  '0.004*"time" + 0.004*"neural" + 0.003*"algorithm" + 0.003*"network" + '
  '0.003*"log" + 0.003*"problem"'),
 (3,
  '0.006*"learning" + 0.005*"algorithm" + 0.004*"model" + 0.004*"time" + '
  '0.004*"neural" + 0.003*"data" + 0.003*"using" + 0.003*"training" + '
  '0.003*"two" + 0.003*"network"'),
 (4,
  '0.005*"learning" + 0.005*"data" + 0.004*"time" + 0.004*"model" + '
  '0.004*"algorithm" + 0.004*"network" + 0.004*"one" + 0.003*"number" + '
  '0.003*"figure" + 0.003*"neural"'),
 (5,
  '0.007*"learning" + 0.005*"algorithm" + 0.005*"data" + 0.004*

In [ ]:
doc_lda = lda_model[corpus]

### I KNOW THIS  IS STILL CONFUSING ! LETS VISUALISE IT with 'pyLDAvis'

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
ldavis_file_path = os.path.join("/kaggle/working/ldavis_prepared_"+str(num_topics))

In [ ]:
if 1==1: # Just to make statement True , !
    ldavis_prepared =  pyLDAvis.gensim.prepare(lda_model,corpus,id2word)
    
    with open(ldavis_file_path,"wb") as f:
        pickle.dump(ldavis_prepared,f)

/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
with open(ldavis_file_path,"rb") as f:
    ldavis_prepared = pickle.load(f)
    


In [ ]:
pyLDAvis.save_html(ldavis_prepared, '/kaggle/working/ldavis_prepared_'+ str(num_topics) +'.html')

In [ ]:
ldavis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.003948 -0.000068       1        1  23.670450
9     -0.003574 -0.002821       2        1  17.481672
6     -0.002810  0.003902       3        1  12.600985
0     -0.001590  0.001458       4        1  11.183852
4      0.000058 -0.002881       5        1   7.104307
8      0.008111 -0.008401       6        1   6.911516
1      0.002451  0.006902       7        1   6.734650
3      0.004819  0.004387       8        1   6.110521
5      0.003910  0.001305       9        1   4.282548
2     -0.007425 -0.003782      10        1   3.919500, topic_info=          Term         Freq        Total Category  logprob  loglift
387   learning  1408.000000  1408.000000  Default  30.0000  30.0000
19   algorithm  1030.000000  1030.000000  Default  29.0000  29.0000
730       time   866.000000   866.000000  Default  28.0000  28.0000
404        log   658.000000   658.000000  Default  27.0000  27.0000
487        one   778.000000   778.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
952      error    20.178733   546.726028  Topic10  -6.1135  -0.0601
250     figure    21.839052   699.869055  Topic10  -6.0345  -0.2280
277      given    19.331899   526.475510  Topic10  -6.1564  -0.0652
739   training    18.568810   745.322115  Topic10  -6.1967  -0.4531
23        also    17.772664   581.852794  Topic10  -6.2405  -0.2493

[857 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
3703       1  0.210970  abstention
3703       2  0.253164  abstention
3703       3  0.105485  abstention
3703       4  0.063291  abstention
3703       5  0.042194  abstention
...      ...       ...         ...
10642      7  0.052369         zic
10642      8  0.026184         zic
10642      9  0.026184         zic
10642     10  0.026184         zic
16029      1  0.506662         zsc

[3949 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 10, 7, 1, 5, 9, 2, 4, 6, 3])